In [10]:
from pathlib import Path
import pandas as pd
import numpy as np
import datetime as dt
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [11]:
department = os.listdir('./Выгрузки')

In [12]:
#Рейтинг менеджеров
for dep in tqdm(department):
    #из СПДП вытаскиваем уникальных сотрудников
    spdp = pd.read_excel(f'./Выгрузки/{dep}/СПДП.xlsx', header=3)
    spdp.drop_duplicates(keep='first', inplace=True)
    spdp.dropna(axis='columns', how='all', inplace=True)
    spdp = spdp[spdp['Сотрудник ДП.Контрагенты доступ.Не учитывать в ЗП'] == 'Нет']
    
    mp = spdp['Сотрудник ДП.ФИО Сотрудника'].unique().tolist()
    df = pd.DataFrame({'МП': mp})

    pf = pd.read_excel(f'./Выгрузки/{dep}/ПФ.xlsx', header=17)
    df = df.merge(pf[['Руководитель', 'План', 'Факт']], how='left', left_on='МП', right_on='Руководитель')

    #замена пустых значение в колонке "План" на 0. Сохраним в df только те строки, где значения в ПЛАН > 0
    df['План'].fillna(0)
    df = df[df['План'] > 0]

    pr = pd.read_excel(f'./Выгрузки/{dep}/Прирост.xlsx', header=5)
    df = df.merge(pr[['Факт за текущий месяц прошлого года', 'Руководитель']], how='left', left_on='МП', right_on='Руководитель')

    nk = pd.read_excel(f'./Выгрузки/{dep}/НК.xlsx', header=4)
    df = df.merge(nk[['Менеджер', 'Факт', 'Сумма реализаций']], how='left', left_on='МП', right_on='Менеджер')

    vk = pd.read_excel(f'./Выгрузки/{dep}/ВК.xlsx', header=4)
    df = df.merge(vk[['Возвращено', 'Сумма реализаций', 'Сотрудник']], how='left', left_on='МП', right_on='Сотрудник')

    df['Выполнение плана, %'] = df['Факт_x'] / df['План']
    df['Прирост по территории, %'] = df['Факт_x'] / df['Факт за текущий месяц прошлого года'] - 1

    df.rename(columns={'Факт_x':'Факт продаж 2023',
                        'Факт за текущий месяц прошлого года':'Факт продаж 2022',
                        'Факт_y':'Количество НК', 
                        'Сумма реализаций_x':'Сумма новых клиентов (тыс. руб)',
                        'Возвращено':'Количество ВК', 
                        'Сумма реализаций_y':'Сумма возвращенных клиентов (тыс. руб)'}, inplace=True)

    #руководители и регион
    df = df.merge(spdp[['Сотрудник ДП.ФИО Сотрудника','Непосредственный руководитель']], how='left', left_on='МП', right_on='Сотрудник ДП.ФИО Сотрудника')

    slovar = pd.read_excel(f'./Выгрузки/{dep}/Словарь руководителей и регионов.xlsx', header=0)
    df = df.merge(slovar, how='left', left_on='Непосредственный руководитель', right_on='Непосредственный руководитель/сервис-менеджер')
    df.rename(columns={'Регион':'Округ'}, inplace=True)

    df.drop_duplicates(subset='МП', keep='first', inplace=True)

    #подтягиваем продажи к новым клиентам
    prod = pd.read_excel(f'./Выгрузки/{dep}/Продажи.xlsx', header=7)
    nk = nk.merge(prod, how='left', left_on='Менеджер', right_on='Контрагент')

    #датафрейм только с менеджерами для подтягивания новых клиентов
    df2 = pd.DataFrame({'МП': mp})
    nk = nk.merge(df2, how='left', left_on='Менеджер', right_on='МП')

    nk['МП'].fillna(method='ffill', inplace=True)
    new_nk = nk.groupby(by='МП', as_index=False).agg(sum)
    new_nk = new_nk[['МП', 'Сумма продажи в руб.']]

    df = df.merge(new_nk, how='left', left_on='МП', right_on='МП')
    df.rename(columns={'Сумма продажи в руб.':'Сумма новых клиентов по разновидности (тыс. руб)'}, inplace=True)

    #возвращенные клиенты
    vk = pd.read_excel(f'./Выгрузки/{dep}/ВК.xlsx', header=4)
    vk = vk.merge(prod, how='left', left_on='Сотрудник', right_on='Контрагент')

    #датафрейм только с менеджерами для подтягивания возвращенных клиентов
    df3 = pd.DataFrame({'МП': mp})
    vk = vk.merge(df2, how='left', left_on='Сотрудник', right_on='МП')

    vk['МП'].fillna(method='ffill', inplace=True)
    new_vk = vk.groupby(by='МП', as_index=False).agg(sum)
    new_vk = new_vk[['МП', 'Сумма продажи в руб.']]

    df = df.merge(new_vk, how='left', left_on='МП', right_on='МП')
    df.rename(columns={'Сумма продажи в руб.':'Сумма возвращенных клиентов по разновидности (тыс. руб)'}, inplace=True)

    df[['Сумма новых клиентов по разновидности (тыс. руб)', 'Сумма возвращенных клиентов по разновидности (тыс. руб)']] = df[['Сумма новых клиентов по разновидности (тыс. руб)',\
                                                                                                                        'Сумма возвращенных клиентов по разновидности (тыс. руб)']].fillna(0)

    df['% НК от факта'] = df['Сумма новых клиентов по разновидности (тыс. руб)'] / df['Факт продаж 2023']
    df['% ВК от факта'] = df['Сумма возвращенных клиентов по разновидности (тыс. руб)'] / df['Факт продаж 2023']

    df['Кол-во дней в командировке'] = ""

    #вычисления последних столбцов
    min_plan = min(df['Выполнение плана, %'])
    max_plan = max(df['Выполнение плана, %'])
    df['Выполнение плана, %, расчет инт. показателя Z преобразованием'] = (df['Выполнение плана, %'] - min_plan) / (max_plan - min_plan) * 2

    min_pr = min(df['Прирост по территории, %'])
    max_pr = max(df['Прирост по территории, %'])
    df['Прирост по территории, %, расчет инт. показателя Z преобразованием'] = (df['Прирост по территории, %'] - min_pr) / (max_pr - min_pr) * 1.5

    if df['% НК от факта'].sum() != 0:
        min_nk = min(df['% НК от факта'])
        max_nk = max(df['% НК от факта'])
        df['Сумма новых клиентов (тыс. руб), расчет инт. показателя Z преобразованием'] = (df['% НК от факта'] - min_nk) / (max_nk - min_nk) * 0.5
    else:
        df['Сумма новых клиентов (тыс. руб), расчет инт. показателя Z преобразованием'] = 0
      
    min_vk = min(df['% ВК от факта'])
    max_vk = max(df['% ВК от факта'])
    df['Сумма возвращенных клиентов (тыс. руб), расчет инт. показателя Z преобразованием'] = (df['% ВК от факта'] - min_vk) / (max_vk - min_vk) * 0.5

    if df['% ВК от факта'].sum() != 0:
        min_nk = min(df['% ВК от факта'])
        max_nk = max(df['% ВК от факта'])
        df['Сумма возвращенных клиентов (тыс. руб), расчет инт. показателя Z преобразованием'] = (df['% ВК от факта']-min_nk) / (max_nk - min_nk) * 0.5
    else:
        df['Сумма возвращенных клиентов (тыс. руб), расчет инт. показателя Z преобразованием'] = 0

    df['ИТОГО СУММА ИНТЕГРАЛЬНЫХ ПОКАЗАТЕЛЕЙ'] = df['Выполнение плана, %, расчет инт. показателя Z преобразованием']\
                                                + df['Прирост по территории, %, расчет инт. показателя Z преобразованием']\
                                                + df['Сумма новых клиентов (тыс. руб), расчет инт. показателя Z преобразованием']\
                                                + df['Сумма возвращенных клиентов (тыс. руб), расчет инт. показателя Z преобразованием']

    df['РЕЙТИНГ ПРОДАВЦА ОБЩИЙ'] = df['ИТОГО СУММА ИНТЕГРАЛЬНЫХ ПОКАЗАТЕЛЕЙ'].rank(ascending=False, method='min')
    df['РЕЙТИНГ ПО ВЫПОЛНЕНИЮ ПЛАНА'] = df['Выполнение плана, %, расчет инт. показателя Z преобразованием'].rank(ascending=False, method='min')
    df['РЕЙТИНГ ПО ПРИРОСТУ'] = df['Прирост по территории, %, расчет инт. показателя Z преобразованием'].rank(ascending=False, method='min')
    df['РЕЙТИНГ ПО СУММЕ НОВЫХ КЛИЕНТОВ'] = df['Сумма новых клиентов (тыс. руб), расчет инт. показателя Z преобразованием'].rank(ascending=False, method='min')
    df['РЕЙТИНГ ПО СУММЕ ВОЗВРАЩЕННЫХ КЛИЕНТОВ'] = df['Сумма возвращенных клиентов (тыс. руб), расчет инт. показателя Z преобразованием'].rank(ascending=False, method='min')

    df.sort_values(['Непосредственно руководитель (линейник)', 'МП'], ascending=True, inplace=True)

    df2 = df[['МП', 
            'Округ', 
            'Непосредственно руководитель (линейник)', 
            'План', 
            'Выполнение плана, %', 
            'Прирост по территории, %', 
            'Факт продаж 2022', 
            'Факт продаж 2023', 
            'Количество НК',
            'Сумма новых клиентов (тыс. руб)', 
            'Сумма новых клиентов по разновидности (тыс. руб)',
            '% НК от факта',
            'Количество ВК',  
            'Сумма возвращенных клиентов (тыс. руб)', 
            'Сумма возвращенных клиентов по разновидности (тыс. руб)', 
            '% ВК от факта',
            'Кол-во дней в командировке', 
            'РЕЙТИНГ ПРОДАВЦА ОБЩИЙ', 
            'РЕЙТИНГ ПО ВЫПОЛНЕНИЮ ПЛАНА', 
            'РЕЙТИНГ ПО ПРИРОСТУ', 
            'РЕЙТИНГ ПО СУММЕ НОВЫХ КЛИЕНТОВ',
            'РЕЙТИНГ ПО СУММЕ ВОЗВРАЩЕННЫХ КЛИЕНТОВ']]
            
    #поставить 0 вместо пустоты
    df2['Сумма новых клиентов (тыс. руб)'] = df2['Сумма новых клиентов (тыс. руб)'].fillna(0)
    df2['Сумма возвращенных клиентов (тыс. руб)'] = df2['Сумма возвращенных клиентов (тыс. руб)'].fillna(0)

    df2.to_excel(f'./Готовые рейтинги по ДП/{dep}.xlsx', index=False)

#объединение всех файлов в 1
all_r = Path(f'./Готовые рейтинги по ДП')
df3 = (pd.concat([pd.read_excel(f).assign(file_name=f.stem) for f in all_r.glob('*xlsx')], ignore_index=True))

df3.to_excel(f'./Rating_managers.xlsx', index=False)

100%|██████████| 3/3 [00:04<00:00,  1.60s/it]
